In [ ]:

from classes.Grid import *


def run_app():
    root = Tk()
    root.geometry("1000x1000")
    GridWindow(root)
    root.mainloop()


if __name__ == '__main__':

    run_app()



Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-1.json
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
36.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
52.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
68.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
84.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
116.0
116.0
116.0
116.0
116.0
116.0
116.0
116.0
